# Подключение зависимостей

In [1]:
import os
import shutil
import pandas as pd

from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# Подключение Гугл-диска с изображениями для обучения

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Определение констант

In [3]:
FIRST_CLASS_NAME = 'rose'
SECOND_CLASS_NAME = 'sunflower'

NB_IMAGES = 733

TEST_DATA_PORTION = 0.15
VAL_DATA_PORTION = 0.15

DATA_DIR = r'drive/MyDrive/Учеба/ОиРС/Семестр 8/Изображения'
TRAIN_DIR = r'data/train'
TEST_DIR = r'data/test'
VAL_DIR = r'data/val'

IMG_WIDTH, IMG_HEIGHT = 150, 150
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

EPOCHS = 13
BATCH_SIZE = 16

# Определение функции создания каталогов, в которые будут помещены изображения

In [4]:
def create_directory(dir_name):
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
  os.makedirs(dir_name)
  os.makedirs(os.path.join(dir_name, FIRST_CLASS_NAME))
  os.makedirs(os.path.join(dir_name, SECOND_CLASS_NAME))

# Определение функции копирования изображений
На входе источник всех картинок, словарь вида `{целевая папка: количество фото}`<br>
Алгоритм все изображения размещает по целевым директориям в требуемом количестве

In [5]:
def copy_images(source_dir, dest_map):
  for cl in (FIRST_CLASS_NAME, SECOND_CLASS_NAME):
    file_counter = 0
    current_dest = 0
    for address, _, files in os.walk(os.path.join(source_dir, cl)):
      for file in files:
        shutil.copy2(os.path.join(address, file), os.path.join(list(dest_map.keys())[current_dest], cl))
        file_counter += 1
        if file_counter == list(dest_map.items())[current_dest][1]:
          if current_dest < len(dest_map) - 1:
            current_dest += 1
            file_counter = 0

# Определение функции подготовки изображений для обучения
Последовательно создаются папки для картинок, картинки перемещаются в эти папки

In [6]:
def prepare_data():
  create_directory(TRAIN_DIR)
  create_directory(TEST_DIR)
  create_directory(VAL_DIR)
  val_data_number = int(NB_IMAGES * VAL_DATA_PORTION)
  test_data_number = int(NB_IMAGES * TEST_DATA_PORTION)
  print(f'All: {NB_IMAGES}\nVal: {val_data_number}\nTest: {test_data_number}')

  copy_images(DATA_DIR, {
      TRAIN_DIR: NB_IMAGES - val_data_number - test_data_number,
      VAL_DIR: val_data_number,
      TEST_DIR: test_data_number
      })

In [7]:
prepare_data()

All: 733
Val: 109
Test: 109


# Инициализация модели, запуск обучения, сохранение модели в `.h5` файл

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [10]:
train_generator = datagen.flow_from_directory(TRAIN_DIR,
                                              target_size=(IMG_WIDTH, IMG_HEIGHT),
                                              batch_size=BATCH_SIZE,
                                              class_mode='binary')

Found 1030 images belonging to 2 classes.


In [11]:
test_generator = datagen.flow_from_directory(TEST_DIR,
                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                             batch_size=BATCH_SIZE,
                                             class_mode='binary')

Found 269 images belonging to 2 classes.


In [12]:
val_generator = datagen.flow_from_directory(VAL_DIR,
                                            target_size=(IMG_WIDTH, IMG_HEIGHT),
                                            batch_size=BATCH_SIZE,
                                            class_mode='binary')

Found 218 images belonging to 2 classes.


In [13]:
model.fit(train_generator, steps_per_epoch=NB_IMAGES // BATCH_SIZE,
          epochs=EPOCHS, validation_data=val_generator,
          validation_steps=int(NB_IMAGES * VAL_DATA_PORTION) // BATCH_SIZE)

Epoch 1/13
45/45 [==============================] - 25s 519ms/step - loss: 0.5753 - accuracy: 0.6930 - val_loss: 0.3608 - val_accuracy: 0.7812
Epoch 2/13
45/45 [==============================] - 23s 504ms/step - loss: 0.3519 - accuracy: 0.8394 - val_loss: 0.1723 - val_accuracy: 0.9479
Epoch 3/13
45/45 [==============================] - 24s 526ms/step - loss: 0.3062 - accuracy: 0.8915 - val_loss: 0.1446 - val_accuracy: 0.9271
Epoch 4/13
45/45 [==============================] - 22s 488ms/step - loss: 0.3146 - accuracy: 0.8889 - val_loss: 0.2192 - val_accuracy: 0.9062
Epoch 5/13
45/45 [==============================] - 21s 476ms/step - loss: 0.2929 - accuracy: 0.8915 - val_loss: 0.1875 - val_accuracy: 0.9167
Epoch 6/13
45/45 [==============================] - 22s 488ms/step - loss: 0.2615 - accuracy: 0.8972 - val_loss: 0.2080 - val_accuracy: 0.9479
Epoch 7/13
45/45 [==============================] - 24s 531ms/step - loss: 0.1990 - accuracy: 0.9181 - val_loss: 0.1689 - val_accuracy: 0.9375

In [14]:
model.save(f'{FIRST_CLASS_NAME}_{SECOND_CLASS_NAME}_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Построение графиков потерь и качества

In [15]:
pd.options.plotting.backend = 'plotly'

In [16]:
history = pd.DataFrame(model.history.history)

In [17]:
fig = history.plot(
    template='simple_white',
    labels=dict(index='Эпоха', value='Значение', variable='Параметры'))
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)
fig.show()